### MSc in Data Science, AUEB
## Text Analytics, Assignment 5
### Exercise 1

In [1]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Succes

In [2]:
import numpy as np

In [3]:
!pip install transformers[torch]
import transformers
transformers.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00


'4.38.2'

In [4]:
!nvidia-smi

Mon Mar 11 20:25:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load the dataset

In [5]:
!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar xzf review_polarity.tar.gz

--2024-03-11 20:25:25--  https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  6.34MB/s    in 0.5s    

2024-03-11 20:25:26 (6.34 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



In [6]:
import os
from sklearn.datasets import load_files


dataset_path = 'txt_sentoken'
movie_reviews = load_files(container_path = dataset_path, encoding = 'utf-8')


x = movie_reviews.data            #the data
y = movie_reviews.target          #the labels
z = movie_reviews.target_names    #the names of labels

In [7]:
x[:1]

["arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \nparts of this are actually so absurd , that they would fit right in with dogma . \nyes , the film is 

In [8]:
y

array([0, 1, 1, ..., 1, 0, 0])

In [9]:
z

['neg', 'pos']

In [10]:
def average_doc_length(corpus):
    ''' Takes as input a whole corpus
      Returns the average number of words and chars per document '''

    document_word_lengths = [len(doc.split()) for doc in corpus]                            #length of each doc (in words)
    average_doc_length_words = sum(document_word_lengths) / len(document_word_lengths)      #average doc length (in words)

    document_char_lengths = [len(doc) for doc in corpus]                                    #length of each doc (in characters)
    average_doc_length_chars = sum(document_char_lengths) / len(document_char_lengths)      #average doc length (in characters)

    return average_doc_length_words, average_doc_length_chars

In [11]:
print("---Before preprocessing---")
avg_length_words, avg_length_chars = average_doc_length(x)
print("Average Document Length (in words):", avg_length_words)
print("Average Document Length (in characters):", avg_length_chars)

---Before preprocessing---
Average Document Length (in words): 746.3405
Average Document Length (in characters): 3893.002


## Pre-processing

The english stopwords is a package of 179 words that in general, would not help in a sentiment analysis problem. But, since they include terms that are negative, removing them could prove harmful for our case.

e.g. imagine the phrase "I didn't like the film" to end up "like film".

So, the plan is to remove all the stop words that include negative meaning before the preprocessing.

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

From these words, we will decide which ones to keep because in fact they have a meaningful impact in our sentiment analysis problem, as we stated earlier.

In [13]:
set_stop_words = set(stopwords.words('english'))

to_keep_words = ['not', "don't", "aren't", "couldn't", "didn't", "doesn't", "hadn't", "hasn't" , "shouldn't", "haven't", "wasn't", "weren't",  "isn't", "doesn"]
to_keep_words

['not',
 "don't",
 "aren't",
 "couldn't",
 "didn't",
 "doesn't",
 "hadn't",
 "hasn't",
 "shouldn't",
 "haven't",
 "wasn't",
 "weren't",
 "isn't",
 'doesn']

In [14]:
stopwords_updated = set(stopwords.words('english')) - set(to_keep_words)
print(len(stopwords.words('english')))
print(len(to_keep_words))
print(len(stopwords_updated))

179
14
165


In [15]:
from nltk.stem import WordNetLemmatizer
import re
from tqdm.auto import tqdm
import string
nltk.download('wordnet')

def pre_process_text(text):
    ''' Function to preprocess text.
     input: initial text
     output: processed text
     Performs pre-processing methods:
        1. Combination to a single document.
        2. Convertion to lowercase.
        3. Lemmatization and stop words extraction
        4. Punctuation removal
        5. Number removal
        6. Single characters removal
        7. Converting multiple spaces to single ones
        '''
    lemmatizer = WordNetLemmatizer()
    all_docs = []

    single_char = re.compile(r'\s+[a-z]\s+')                                          #6. Remove single characters
    multiple_space= re.compile(r'\s+')                                                 #7. Replace multiple space with a single one

    stopwords_updated = set(stopwords.words('english')) - set(to_keep_words)
    for document in tqdm(x):

        combined_text = ' '.join(text)            #1.Combine in one single document

        combined_text = combined_text.lower()    #2. Convert to lowercase
        combined_text = [lemmatizer.lemmatize(word) for word in document.split() if word not in stopwords_updated]  # 3.Lemmatize and remove stop words


        combined_text = ' '.join(combined_text)

        combined_text = ''.join([char for char in combined_text if char not in string.punctuation])   #4.remove punctuation
        combined_text = ''.join([char for char in combined_text if not char.isdigit()])     #5.remove numbers

        res = single_char.sub(combined_text, '')
        res2 = multiple_space.sub(combined_text, ' ')
        all_docs.append(combined_text)

    return all_docs

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [16]:
processed_text = pre_process_text(x)

processed_text[:1]

  0%|          | 0/2000 [00:00<?, ?it/s]

['arnold schwarzenegger icon action enthusiast  since late s  lately film sloppy oneliner getting worse  hard seeing arnold mr  freeze batman robin  especially say ton ice joke  hey got  million  whats matter  arnold signed another expensive blockbuster  cant compare like terminator series  true lie even eraser  called dark thriller  devil  gabriel byrne  come upon earth  impregnate woman  robin tunney  happens every  year  basically destroy world  apparently god chosen one man  one man jericho cane  arnold   help trusty sidekick  kevin pollack   stop nothing let devil take world  part actually absurd  would fit right dogma  yes  film weak  better blockbuster right  sleepy hollow   make world not enough look like  star film  anyway  definitely doesnt seem like arnold movie  wasnt type film see  sure gave u chuckle well known oneliner  seemed confused character film going  understandable  especially ending changed according source  aside form  still walked  much like past film  im sorry

## Data Augmentation

Our dataset consists of 2000 reviews, as we stated earlier. The size can be considered rather small and classification algorithms may be led to overfitting.

One measure we may take to face this is augmenting the text data. This involves generating new data points by making minor modifications on the existing ones. Here we will use the technique of synonym replacement

**Process**: We split the input sentence into words. For each word, we retrieve synonyms from the 'synsets' function of WordNet. If we find synonyms, we randomly select one and we extract the canonical form of it (lemmatization). Finally, we replace the original word with the lemma if they are different.

In [17]:
import nltk
from nltk.corpus import wordnet
import random


nltk.download('wordnet')


def synonym_replacement(sentence):
    words = sentence.split()
    new_words = words.copy()
    for i in range(len(words)):
        syns = wordnet.synsets(words[i])
        if syns:
            syn = random.choice(syns)
            lemma = syn.lemmas()[0].name()
            new_words[i] = lemma if lemma != words[i] else new_words[i]
    return ' '.join(new_words)


augmented_text = []
augmented_y = []

for i in range(len(processed_text)):
    original_sentence = processed_text[i]
    augmented_sentence = synonym_replacement(original_sentence)
    augmented_text.append(augmented_sentence)
    augmented_y.append(y[i])                        #Append the label for the augmented example
    augmented_text.append(original_sentence)        #Add original sentence as well
    augmented_y.append(y[i])                        #Append the label for the original example


z_augmented = z + z

print("Original data size:", len(processed_text))
print("Augmented data size:", len(augmented_text))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original data size: 2000
Augmented data size: 4000


## Splitting into training set (70%), development set (15%) and test set (15%)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(augmented_text, augmented_y, test_size=0.3, random_state=17)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=25)

training_text = ' '.join(X_train)                    #Flatten into a single string
development_text = ' '.join(X_dev)
test_text = ' '.join(X_test)

training_words = training_text.split()
development_words = development_text.split()
test_words = test_text.split()

training_vocab = set(training_words)
development_vocab = set(development_words)
test_vocab = set(test_words)

In [19]:
print("Training set size (in documents): ", len(y_train))
print("Development set size (in documents): ", len(y_dev))
print("Test set size (in documents): ", len(y_test))
print("Full size (sanity check): ", len(y_train) + len(y_dev) + len(y_test))
print("---------------------------------")
print("Training vocabulary size (in words): " , len(training_vocab))
print("Development vocabulary size (in words): ", len(development_vocab))
print("Test vocabulary size (in words): ", len(test_vocab))
print("Full vocabulary size (in words): ", len(training_vocab) + len(development_vocab) + len(test_vocab))

Training set size (in documents):  2800
Development set size (in documents):  600
Test set size (in documents):  600
Full size (sanity check):  4000
---------------------------------
Training vocabulary size (in words):  46050
Development vocabulary size (in words):  25104
Test vocabulary size (in words):  25425
Full vocabulary size (in words):  96579


In [20]:
import spacy

nlp = spacy.load('en_core_web_sm',disable=["tagger", "parser","ner"])
nlp.add_pipe('sentencizer')

## Use spaCy for sentence splitting & tokenization

In [21]:
def tokenize_sent(x):

  X_tokenized = []
  for idx in tqdm(range(len(x))):
    doc = nlp(x[idx])
    tokens = []
    for sent in doc.sents:
      for tok in sent:
        if '\n' in tok.text or "\t" in tok.text or "--" in tok.text or "*" in tok.text or tok.text.lower() in stopwords_updated or tok.text in string.punctuation or all(x in string.punctuation for x in tok.text):
          continue
        if tok.text.strip():
          tokens.append(tok.text.replace('"',"'").strip().lower())
    X_tokenized.append(tokens)
  return X_tokenized

In [22]:
X_train_tokenized = tokenize_sent(X_train)
X_dev_tokenized = tokenize_sent(X_dev)
X_test_tokenized = tokenize_sent(X_test)

  0%|          | 0/2800 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

In [23]:
# Get mean and std of sequence length
train_mean = np.mean([len(x) for x in X_train_tokenized])
train_std = np.std([len(x) for x in X_train_tokenized])
print("Mean of sequence length on training set:", train_mean)
print("Standard deviation of sequence length on training set:", train_std)

print("-----------------------------------------------------------")

dev_mean = np.mean([len(x) for x in X_dev_tokenized])
dev_std = np.std([len(x) for x in X_dev_tokenized])
print("Mean of sequence length on development set:", dev_mean)
print("Standard deviation of sequence length on development set:", dev_std)

print("-----------------------------------------------------------")

test_mean = np.mean([len(x) for x in X_test_tokenized])
test_std = np.std([len(x) for x in X_test_tokenized])
print("Mean of sequence length on test set:", test_mean)
print("Standard deviation of sequence length on test set:", test_std)

Mean of sequence length on training set: 350.7057142857143
Standard deviation of sequence length on training set: 151.6876460604058
-----------------------------------------------------------
Mean of sequence length on development set: 350.74
Standard deviation of sequence length on development set: 150.87555269161402
-----------------------------------------------------------
Mean of sequence length on test set: 359.49
Standard deviation of sequence length on test set: 155.18122706478815


In [24]:
X_train_clean = [" ".join(x) for x in X_train_tokenized]
X_dev_clean = [" ".join(x) for x in X_dev_tokenized]
X_test_clean = [" ".join(x) for x in X_test_tokenized]

In [25]:
%%capture
!pip install transformers
!pip install datasets
!pip install accelerate

In [26]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

d = {'train': Dataset.from_dict({'label':y_train, 'text':X_train_clean}),
     'dev': Dataset.from_dict({'label':y_dev, 'text':X_dev_clean}),
     'test': Dataset.from_dict({'label': y_test, 'text':X_test_clean})
     }

reviews_dataset = DatasetDict(d)

In [27]:
reviews_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2800
    })
    dev: Dataset({
        features: ['label', 'text'],
        num_rows: 600
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 600
    })
})

## Bert for text classification

In [28]:
from transformers import DistilBertTokenizer

model = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [29]:
max_length = tokenizer.model_max_length

def bpe_tokenization(dataset):
  return tokenizer(dataset['text'],
                   truncation = True,
                   padding = 'max_length',
                   max_length = max_length)

In [30]:
tokenized_reviews_dataset = reviews_dataset.map(bpe_tokenization, batched = True)

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [31]:
tokenized_reviews_dataset['train'][1]

{'label': 0,
 'text': 'rat roentgen firm ferocity terminology drug habit nakedness sex leading wesley snipe anne sagittarius michael chaykin donald sutherland marie matiko running prison_term minute steer christian duguay never amply understand wesley attack career largely good movie one well movie entitle sword distinctive action movie boiling point passenger fresh movie artwork war call rise sun art war induce basically semiremake plat different cast impel laughably improbable thriller high style low substance wesley snipe play jam bondish type secret agent working u newton chinese ambassador kill track killer trying discovery happen cause_of_death anne sagittarius play bos character person u north lease attack track killer well sharpshoot mistakenly frame cause_of_death illegitimate kidnapping chinese woman help along way lead finale dumb unintelligent incredibly speechless stunt dialogue act ruin movie wesley sharpshoot nt one favored actor fact character play every movie character

In [32]:
tokenizer.convert_ids_to_tokens(tokenized_reviews_dataset['train'][1]['input_ids'])[:10]

['[CLS]',
 'rat',
 'roe',
 '##nt',
 '##gen',
 'firm',
 'fe',
 '##rocity',
 'terminology',
 'drug']

## Pad sequences

In [33]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

## Load pre-trained model

In [34]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer

tr_model = DistilBertForSequenceClassification.from_pretrained(model, num_labels = 2).to('cuda')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define evaluation metrics

In [35]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)


    acc = accuracy_score(labels, preds)


    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, zero_division = 0)




    metrics_per_class = {}
    for i in range(len(precision)):
        metrics_per_class['class', z[i]] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1': f1[i]
        }
    print(metrics_per_class)

    return {
        'val_accuracy': acc,
        #'metrics_per_class': metrics_per_class,

    }


In [36]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.9 MB/s eta 0:00:00


## Hyperparameter search

In [40]:
import optuna
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score

def model_init():
    return DistilBertForSequenceClassification.from_pretrained(model)


def objective(trial):

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 3)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])



    training_args = TrainingArguments(
        output_dir=f"./results2/trial-{trial.number}",
        evaluation_strategy="epoch",
        save_strategy='epoch',
        per_device_eval_batch_size=16,
        logging_dir="./logs",
        logging_steps=100,
        weight_decay=0.01,
        save_steps=-1,
        warmup_steps=500,
        report_to="none",
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        load_best_model_at_end=True,
        per_device_train_batch_size=per_device_train_batch_size
    )


    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=tokenized_reviews_dataset['train'],
        eval_dataset=tokenized_reviews_dataset['dev'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )



    trainer.train()

    eval_results = trainer.evaluate()


    return eval_results['eval_val_accuracy']


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)




[I 2024-03-11 20:55:06,746] A new study created in memory with name: no-name-ff66eb8f-cc97-4e0c-8201-9f38f4b139de
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Val Accuracy
1,No log,0.686930,0.583333


{('class', 'neg'): {'precision': 0.7956989247311828, 'recall': 0.24262295081967214, 'f1': 0.371859296482412}, ('class', 'pos'): {'precision': 0.5443786982248521, 'recall': 0.9355932203389831, 'f1': 0.688279301745636}}


[I 2024-03-11 20:57:54,095] Trial 0 finished with value: 0.5833333333333334 and parameters: {'learning_rate': 3.6476444550369784e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.5833333333333334.


{('class', 'neg'): {'precision': 0.7956989247311828, 'recall': 0.24262295081967214, 'f1': 0.371859296482412}, ('class', 'pos'): {'precision': 0.5443786982248521, 'recall': 0.9355932203389831, 'f1': 0.688279301745636}}


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Val Accuracy
1,0.694000,0.676406,0.603333
2,0.542200,0.329649,0.871667


{('class', 'neg'): {'precision': 0.7723577235772358, 'recall': 0.3114754098360656, 'f1': 0.4439252336448598}, ('class', 'pos'): {'precision': 0.559748427672956, 'recall': 0.9050847457627119, 'f1': 0.6917098445595855}}
{('class', 'neg'): {'precision': 0.8352941176470589, 'recall': 0.9311475409836065, 'f1': 0.8806201550387596}, ('class', 'pos'): {'precision': 0.9192307692307692, 'recall': 0.8101694915254237, 'f1': 0.8612612612612612}}


[I 2024-03-11 21:17:03,559] Trial 4 finished with value: 0.8716666666666667 and parameters: {'learning_rate': 2.3301741810228525e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 16}. Best is trial 2 with value: 0.8833333333333333.


{('class', 'neg'): {'precision': 0.8352941176470589, 'recall': 0.9311475409836065, 'f1': 0.8806201550387596}, ('class', 'pos'): {'precision': 0.9192307692307692, 'recall': 0.8101694915254237, 'f1': 0.8612612612612612}}


## Best model

In [41]:
best_hyperparameters = study.best_params

learning_rate = best_hyperparameters["learning_rate"]
num_train_epochs = best_hyperparameters["num_train_epochs"]
per_device_train_batch_size = best_hyperparameters["per_device_train_batch_size"]


In [42]:
print("----------- Best hyperparameters -------------")
print("Learning Rate:", learning_rate)
print("Number of training epochs:", num_train_epochs)
print("Train batch size per device:", per_device_train_batch_size)


print("\n")
best_accuracy = study.best_value
print("Best accuracy:", best_accuracy)




----------- Best hyperparameters -------------
Learning Rate: 2.1098680383101906e-05
Number of training epochs: 2
Train batch size per device: 8


Best accuracy: 0.8833333333333333


In [43]:
best_trial = study.best_trial.number
best_trial

2

In [44]:

best_model = DistilBertForSequenceClassification.from_pretrained(
    f"./results2/trial-{best_trial}/checkpoint-350"
)

In [45]:
def get_prediction(text):
  inputs = tokenizer(text, truncation=True, return_tensors='pt')

  outputs = best_model(**inputs)

  probs = outputs[0].sigmoid()

  return z[probs.argmax()]

In [46]:
text1 = 'I do not like this movie, it is not pretty cool'
text2 = 'With his second foray into Frank Herbert\'s universe, Denis Villeneuve has once again crafted a cinematic juggernaut designed to be seen on the biggest screen and heard through the most powerful sound system possible.'
text3 = 'You know Villeneuve will get the spectacle right. The question is about the human drama… It almost all connected in Part Two. I doubt it can catch up to the expectations'

print(get_prediction(text1))
print(get_prediction(text2))
print(get_prediction(text3))

neg
pos
neg


## Experimental results by prompting an LLM (ChatGPT)

In [47]:
import random
random.seed(1)

num_test_examples = 20
selected_indexes = random.sample(range(len(X_test_clean)), num_test_examples)

selected_test_examples = [X_test_clean[i] for i in selected_indexes]
selected_labels = [y_test[i] for i in selected_indexes]

for idx, example in enumerate(selected_test_examples, start=1):
    print(f"Test Example {idx}: {example}")

demonstrators_examples = selected_test_examples[:10]
demonstrators_labels = selected_labels[:10]

test_examples = selected_test_examples[10:20]
test_labels = selected_labels[10:20]

Test Example 1: viking v digest nt lineup monday night football rather two pit effect battle death thorium warrior film dramatically flat gratuitously bloodstained free base michael crichton book eater dead viking saga sample provoke mysticism fabled norwegian glorious struggle fight strength respect would finally etch place history among great warrior ever pick sword fortunately viking however war ability not gawky film antonio bandaras ahmed travel ambassador accompany friend omar shariff cameo finally come across humble viking village see viking highly proud group whose great potency fortitude laugh heartily delight arrogance sing song battle celebration stop messenger male_child arrive tell village leader capital evil threaten country help want cursorily aged fortunetelling hag arrive brief incantation proclaim warrior one not scandinavian blood want guarantee victory personally always curiosity people believe prophet search haggardly verge insanity however work_force eagerly accep

In [48]:
demonstrators_labels[:10]                                     # The labels of the demonstrators

[0, 0, 0, 1, 1, 0, 1, 1, 0, 0]

In [49]:
llm_predictions = [1, 1, 0, 1, 1, 1, 0, 0, 1, 0]          # The predicted labels of the test examples by ChatGPT

In [50]:
test_labels                                          # The labels of the test examples

[0, 1, 0, 1, 1, 1, 1, 0, 1, 0]

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(test_labels, llm_predictions)
precision = precision_score(test_labels, llm_predictions)
recall = recall_score(test_labels, llm_predictions)
f1 = f1_score(test_labels, llm_predictions)

print("Evaluation Metrics:")
print("-------------------")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Evaluation Metrics:
-------------------
Accuracy: 0.80
Precision: 0.83
Recall: 0.83
F1-Score: 0.83
